# The notebook is used to fetch the Top-100 of actors and actressess of all time from IMDB top list

1. It saves the output into a csv file (list_act.csv) with the following columns: rank, name, imdb_id
2. It retrieves images from imdb based on provided list_act.csv and saves them to the output folder

In [ ]:
# pip install cinemagoer

In [2]:
# Scraping from IMDB

from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import imdb  # Comes from cinemagoer package
from imdb import helpers as hlp
import requests
import os
import unicodedata


### 1. Scraping of the best actors and actresses of all time 

In [ ]:
# Url list of top 100 actresses and actors from IMDB

url_list = ["https://www.imdb.com/list/ls063784435/", "https://www.imdb.com/list/ls050274118/"] # Url for top 100 actors and top100 actressess


# Dictionary with the keys that will be used to store the data in a csv file
act_dict = {
    "rank":[],
    "name":[],
    "imdb_id":[]
    }

for url in url_list:
    response = get(url) 

    # Parse the content of current iteration of request
    page_html = BeautifulSoup(response.text, 'html.parser')
        
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-detail')

    for container in movie_containers:
        act_dict["rank"].append(container.find('h3', class_ = 'lister-item-header').find('span', class_ = 'lister-item-index unbold text-primary').text.split(".")[0])
        act_dict["name"].append(container.find('h3', class_ = 'lister-item-header').find("a").text.strip())
        act_dict["imdb_id"].append(container.find('h3', class_ = 'lister-item-header').find("a")["href"].split("/")[2])
   
df_act = pd.DataFrame(act_dict)
df_act.to_csv("../raw_data/list_act.csv")


In [3]:
# Read the list_act.csv file to a dataframe
df_act = pd.read_csv("../raw_data/list_act.csv")


### 2. Retrieval of images based on scraped names and saving to a provided path

In [ ]:
ia = imdb.Cinemagoer()

folder_name = "output_imdb_top100"

# Remove diacritics from name
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

for actor in df_act["imdb_id"]:
    actor_id = int(actor.strip("nm"))
    person = ia.get_person(actor_id)
    jpg_path = hlp.resizeImage(person["headshot"],crop="0.2")

    person_name = "_".join(strip_accents(person["name"].lower()).split())+".jpg"

    target_path = os.path.join(os.getcwd(),"..","raw_data", folder_name, person_name)

    original = jpg_path
    target = target_path

    url = jpg_path
    
    # This statement requests the resource at
    # the given link, extracts its contents
    # and saves it in a variable
    data = requests.get(url).content
    
    # Opening a new file named img with extension .jpg
    # This file would store the data of the image file
    f = open(target_path,'wb')
    
    # Storing the image data inside the data variable to the file
    f.write(data)
    f.close()
 